In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
current_folder_path = os.getcwd()
parent_folder_path = os.path.dirname(current_folder_path)
data_save_path = os.path.join(parent_folder_path, 'data')

In [3]:
df = pd.read_excel('Multi-faults_Validation.xlsx')
df.to_pickle("test_df")

In [9]:
df['label'] = df['label'].str.replace(' ', '')


In [15]:
# Split the dataset based on labels
labels = ['Bias+Drift', 'Bias+Gain', 'Bias+Precisiondegradation', 'Bias',
       'Bias+Outliers', 'Drift+Gain', 'Drift+Precisiondegradation',
       'Drift', 'Drift+Outliers', 'Gain+Precisiondegradation', 'Gain',
       'Gain+Outliers', 'Precisiondegradation',
       'Precisiondegradation+Outliers']

datasets = {}
dataframes =[]

for label in labels:
    datasets[label] = df[df['label'] == label]

    # Rename the dataset as df_labelname
    globals()[f'df_{label}'] = datasets[label]
    dataframes.append(f'df_{label}')
    # Optional: Save the dataset to a separate CSV file
    #datasets[label].to_csv(data_save_path+ "/" +f'df_{label}.csv', index=False)

In [11]:
def split_column_into_rows(df, num_columns, overlapping_ratio):
    values = df['value'].to_numpy()  # Get the values from the specified column as a NumPy array

    # Calculate the number of overlapping elements between adjacent rows
    overlapping_elements = int(num_columns * overlapping_ratio / 100)

    # Calculate the total number of elements in each row, considering the overlapping elements
    row_elements = num_columns - overlapping_elements

    # Calculate the number of rows needed
    num_rows = (len(values) + row_elements - 1) // row_elements

    # Calculate the required number of values to fill the last row
    last_row_fill = (num_rows * num_columns) - len(values)

    # Extend the values array with zeros if necessary to ensure even splitting
    values = np.pad(values, (0, last_row_fill), mode='constant', constant_values=0)

    # Reshape the values array into the desired shape with overlapping
    new_values = np.zeros((num_rows, num_columns))
    for i in range(num_rows):
        start = i * row_elements
        end = start + num_columns
        new_values[i] = values[start:end]

    # Convert the new array into a DataFrame
    new_df = pd.DataFrame(new_values)

    label_name = df.at[0, 'label']
    new_df.insert(0,'label',label_name)

    return new_df

In [17]:
test_data = pd.DataFrame()
for key in datasets.keys():
    # Access the dataframe using the key
    df = datasets[key]
    df = df.reset_index(drop=True, inplace=False)
    df = split_column_into_rows(df,150,10)
    df = df.drop(df.index[-1])
    test_data = pd.concat([test_data,df])

test_data = test_data.reset_index(drop=True, inplace=False)

print(test_data.shape)

(4100, 151)


In [19]:
test_labels = test_data['label'].values

In [20]:
test_data_values = test_data.drop(columns={'label'})

In [ ]:
test_data_values.to_pickle('test_data_values')
test_labels = pd.DataFrame(test_labels)
test_labels.to_pickle('test_labels')